In [2]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np


In [3]:
# Use pandas to read the CSV file as a dataframe
df = pd.read_csv("moons400.csv")
# The y values are those labelled 'Class': extract their values
y = df['Class'].values

# The x values are all other columns
del df['Class']    # drop the 'Class' column from the dataframe
X = df.as_matrix() # convert the remaining columns to a numpy array
# Some examples of working with the data, to look at rows/columns
print ("len(X):", len(X))            # outer array: one per sample
print ("len(X[0]):", len(X[0]))      # each inner array is the attributes of one sample
print ("len(X[:,0]):", len(X[:,0]))  # select column 0 from array

# np.shape returns all dimensions of the array
(nsamples, nattribs) = np.shape(X)
print ("X: nsamples =", nsamples, ", nattribs =", nattribs)

len(X): 400
len(X[0]): 2
len(X[:,0]): 400
X: nsamples = 400 , nattribs = 2


In [4]:
datasubset_x = X[0:1]
(dsamples, dattribs) = np.shape(datasubset_x)
print("datasubset: dsamples =", dsamples, ", dattribs =", dattribs)
print(datasubset_x)


datasubset: dsamples = 1 , dattribs = 2
[[ 2.07106946  0.41152931]]


In [5]:
datasubset_y = y[0:1]
print(datasubset_y)

[1]


In [6]:
#layer 1 weights
W11_1 = -0.1
W12_1 = -0.1
W21_1 = -0.1
W22_1 = -0.1
#layer 1 bias
b1_1 = 1
b2_1 = 0.9
#layer 2 weights
W11_2 = 0.1
W12_2 = 0.1
#layer 2 bias
b_2 = 0.5


In [15]:
# a1_2 = f(w11_1*x1 + w12_1*x2 +b1_1 )
#print(datasubset_x[0,0])

a1_2 = (W11_1 * datasubset_x[0,0] + W12_1 * datasubset_x[0,1] + b1_1 * 1)
print("a1_2: ",a1_2)

a1_2 = 1/1 - np.exp(a1_2)
print("a1_2exp: ",a1_2)

a2_2 = (W21_1 * datasubset_x[0,0] + W22_1 * datasubset_x[0,1] + b2_1 * 1)
print("a2_2: ",a2_2)
a2_2 = 1/1 - np.exp(a2_2)
print("a2_2exp: ",a2_2)


a1_3 = (W11_2 * a1_2 + W12_2 * a2_2 + b_2 * 1 )
print("a1_3: ",a1_3)
#hW,b(X) = a1_3
a1_3 = 1/1 - np.exp(a1_3)
print("a1_3exp: ",a1_3)


a1_2:  0.751740122949
a1_2exp:  -1.12068706395
a2_2:  0.651740122949
a2_2exp:  -0.918877007408
a1_3:  0.296043592864
a1_3exp:  -0.344528767414


In [13]:
# Total Error SSE = 1/2(Y-YP)^2
sse = 1/2 * np.power((datasubset_y - a1_3), 2)
print("SSE: ",sse)

# ∂SSE/∂a = – (Y-YP)
# ∂SSE/∂b = – (Y-YP)X

SSE:  [ 0.9038788]


In [ ]:
#The Backwards back propagation
